# Step 2: Viterbi algorithm

We begin by reloading the Data_Munging file

In [311]:
%run 'Data_Munging.ipynb'

<class 'pandas.tslib.Timedelta'>
61.0


In [312]:
transitionDistribution

,0,1,4,5,10,13,15,17
0,0.965755,0.000974,0.212329,0.095041,0.001033,0.222222,0.014493,0.475
1,0.006473,0.998539,0.010274,0.000000,0.000000,0.000000,0.000000,0.000
4,0.014408,0.000398,0.609589,0.000000,0.002928,0.000000,0.005797,0.000
5,0.003550,0.000000,0.020548,0.904959,0.000000,0.000000,0.000000,0.000
10,0.003132,0.000000,0.113014,0.000000,0.995867,0.000000,0.000000,0.000
13,0.002297,0.000000,0.020548,0.000000,0.000172,0.777778,0.000000,0.025
15,0.001462,0.000089,0.010274,0.000000,0.000000,0.000000,0.965217,0.000
17,0.002923,0.000000,0.003425,0.000000,0.000000,0.000000,0.014493,0.500


In [313]:
observationDistribution

,0,1,4,5,10,13,15,17
1,0.023805,0.001284,0.000000,0.000000,0.000000,0.088889,0.092754,0.025
5,0.098350,0.002744,0.041096,0.859504,0.001378,0.000000,0.000000,0.000
6,0.123617,0.001859,0.386986,0.066116,0.000000,0.011111,0.000000,0.025
7,0.008979,0.000000,0.000000,0.000000,0.000000,0.011111,0.002899,0.075
8,0.170390,0.000177,0.020548,0.000000,0.000000,0.222222,0.142029,0.675
9,0.053456,0.000266,0.003425,0.000000,0.000000,0.055556,0.156522,0.000
12,0.065776,0.985172,0.037671,0.000000,0.000000,0.000000,0.002899,0.025
13,0.031948,0.002966,0.000000,0.004132,0.000000,0.000000,0.017391,0.050
14,0.190436,0.001903,0.345890,0.066116,0.000431,0.077778,0.005797,0.025
17,0.029651,0.000266,0.000000,0.000000,0.000000,0.122222,0.144928,0.050


In [314]:
initialDistribution = Series([0,0,0,0,1,0,0,0], index = [0,1,4,5,10,13,15,17])
initialDistribution

0     0
1     0
4     0
5     0
10    1
13    0
15    0
17    0
dtype: int64

In [315]:
sensorLabel

1              Microwave
5       Hall-Toilet door
6     Hall-Bathroom door
7          Cups cupboard
8                 Fridge
9        Plates cupboard
12            Front door
13            Dishwasher
14           ToiletFlush
17               Freezer
18         Pans Cupboard
20       Washing machine
23    Groceries Cupboard
24     Hall-Bedroom door
dtype: object

In [316]:
observation = Series([24,24,5,6,14,14,6,9,9,8,8,1,23,1,23,13,9,18,7,9,5,5,12,12,8,23,8,6,14,6,18,18,18,12,18,8,8,7,7,7,8,8,6,6,14,6])
observation

0     24
1     24
2      5
3      6
4     14
5     14
6      6
7      9
8      9
9      8
10     8
11     1
12    23
13     1
14    23
15    13
16     9
17    18
18     7
19     9
20     5
21     5
22    12
23    12
24     8
25    23
26     8
27     6
28    14
29     6
30    18
31    18
32    18
33    12
34    18
35     8
36     8
37     7
38     7
39     7
40     8
41     8
42     6
43     6
44    14
45     6
dtype: int64

In [317]:
observation[5]

14

In [318]:
initialDistribution

0     0
1     0
4     0
5     0
10    1
13    0
15    0
17    0
dtype: int64

In [319]:
activityLabel

0               nothing
1           leave house
4            use toilet
5           take shower
10            go to bed
13    prepare breakfast
15       prepare dinner
17            get drink
dtype: object

In [320]:
def Viterbi(observation, activityLabel, initialDistribution, observationDistribution, transitionDistribution):
    probabilityTable = pd.DataFrame(0.0, index = activityLabel.index, columns = observation.index)
    path = {}
    for activityIndex in activityLabel.index.values:
        probabilityTable[0][activityIndex] = initialDistribution[activityIndex] * observationDistribution[activityIndex][observation[0]]
        path[activityIndex] = [activityIndex]
    
    for observationIndex in range(1,len(observation)):
        newPath = {}
        for activityIndex in activityLabel.index.values:
            (proba, previousActivityInd) = max((probabilityTable[observationIndex-1][previousActivityIndex]*transitionDistribution[previousActivityIndex][activityIndex]*observationDistribution[activityIndex][observation[observationIndex]], previousActivityIndex) for previousActivityIndex in activityLabel.index.values )
            #print proba 
            probabilityTable[observationIndex][activityIndex] = proba
            newPath[activityIndex] = path[previousActivityInd] + [activityIndex]
        path = newPath
    
    (finalProba, finalActivityIndex) = max((probabilityTable[len(observation)-1][activityIndex], activityIndex) for activityIndex in activityLabel.index.values )
    return finalProba, path[finalActivityIndex]
        
        
        

Now we will create the testing set:

In [321]:
day25Sensor = [sensorData['Start time'][x].day == 25 for x in range(1,len(sensorData))]
day25Sensor = day25Sensor + [False]
day25SensorData = sensorData[day25Sensor]
day25SensorData

,Start time,End time,ID,Val
1,2008-02-25 00:20:14,2008-02-25 00:22:57,24,1
2,2008-02-25 09:33:41,2008-02-25 09:33:42,24,1
3,2008-02-25 09:36:43,2008-02-25 09:37:04,5,1
4,2008-02-25 09:37:20,2008-02-25 09:37:23,6,1
5,2008-02-25 09:37:51,2008-02-25 09:37:52,14,1
6,2008-02-25 09:37:55,2008-02-25 09:37:56,14,1
7,2008-02-25 09:37:58,2008-02-25 09:38:01,6,1
8,2008-02-25 09:49:27,2008-02-25 09:49:28,9,1
9,2008-02-25 09:49:31,2008-02-25 09:49:38,9,1
10,2008-02-25 09:49:39,2008-02-25 09:49:44,8,1


In [322]:
day25Activity = [activityData['Start time'][x].day == 25 for x in range(1,len(activityData))]
day25Activity = day25Activity + [False]
day25ActivityData = activityData[day25Activity]
day25ActivityData

,Start time,End time,ID
1,2008-02-25 00:22:46,2008-02-25 09:34:12,10
2,2008-02-25 09:37:17,2008-02-25 09:38:02,4
3,2008-02-25 09:49:23,2008-02-25 09:53:28,13
4,2008-02-25 10:02:28,2008-02-25 10:12:42,5
5,2008-02-25 10:19:06,2008-02-25 16:55:38,1
6,2008-02-25 17:00:31,2008-02-25 17:01:34,4
7,2008-02-25 17:54:55,2008-02-25 17:55:58,17
8,2008-02-25 18:31:54,2008-02-25 18:33:38,4
9,2008-02-25 19:40:26,2008-02-25 20:22:58,15
10,2008-02-25 20:23:12,2008-02-25 20:23:35,17


In [323]:
testingFeatureMatrix, testingLabel = convert2LastFiredFeatureMatrix(day25SensorData,day25ActivityData,60)

In [324]:
testingFeatureMatrix

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,1382.0,1383.0,1384.0,1385.0,1386.0,1387.0,1388.0,1389.0,1390.0,1391.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,1,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [325]:
t = testingFeatureMatrix.ix[:,:10]
t

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
1,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0


In [387]:
testingObservation = Series(0, index = testingFeatureMatrix.columns)
for i in range(len(testingObservation)): 
    r = np.where(testingFeatureMatrix[i] == 1)
    result = testingFeatureMatrix.index[r[0]]
    testingObservation[i] = result[0]

numOfObservation = len(testingObservation)
obs = testingObservation[:numOfObservation]
validatedPath = testingLabel[:numOfObservation]

In [ ]:
proba, path = Viterbi(obs, activityLabel, initialDistribution, observationDistribution, transitionDistribution)


In [ ]:
#error rate
errorRate = ( sum(validatedPath != path)/float(len(validatedPath)) ) * 100 
errorRate

In [ ]:
proba * 100